# Data Cleaning

### Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [111]:
player_info = pd.read_csv('./Data/nfl_draft_prospects.csv')
draft = pd.read_csv('./Data/nfl_draft_profiles.csv')
draft2022 = pd.read_excel('./Data/reports2022.xlsx',index_col=0)

In [112]:
draft2022

,player_name_x,draft_year,pick,text1
6034,Travon Walker,2022,1,Walker is an EDGE defender with an excellent c...
5878,Aidan Hutchinson,2022,2,Hutchinson is a notch below elite in terms of ...
6002,Derek Stingley,2022,3,Stingley is a physical press corner with great...
5846,Sauce Gardner,2022,4,Gardner is the best press-man cornerback in th...
6013,Kayvon Thibodeaux,2022,5,Thibodeaux has the explosive first step to get...
...,...,...,...,...
5858,Jeff Gunter,2022,252,Gunter is a versatile edge defender who lines ...
5849,Trenton Gill,2022,255,Gill has a strong leg. He led the ACC in punt ...
5918,Jesse Luketa,2022,256,Luketa is a relentless run-defender who has go...
5870,Marquis Hayes,2022,257,Hayes is a mauler who plays with good leverage...


In [113]:
scout2 = draft.merge(player_info,how='left',on='player_id')

In [114]:
scout2.columns

Index(['player_id', 'guid_x', 'alt_player_id', 'player_name_x', 'position_x',
       'pos_abbr_x', 'weight_x', 'height_x', 'player_image_x', 'link_x',
       'school_logo', 'school_x', 'school_abbr_x', 'school_name_x', 'pos_rk_x',
       'ovr_rk_x', 'grade_x', 'text1', 'text2', 'text3', 'text4', 'draft_year',
       'player_name_y', 'position_y', 'pos_abbr_y', 'school_y',
       'school_name_y', 'school_abbr_y', 'link_y', 'pick', 'overall', 'round',
       'traded', 'trade_note', 'team', 'team_abbr', 'team_logo_espn', 'guid_y',
       'weight_y', 'height_y', 'pos_rk_y', 'ovr_rk_y', 'grade_y',
       'player_image_y'],
      dtype='object')

In [115]:
add2022 = scout2.append(draft2022)

C:\Users\capta\AppData\Local\Temp\ipykernel_16320\3422529145.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  add2022 = scout2.append(draft2022)


In [116]:
scout = add2022[['player_name_x','pos_abbr_x','weight_x','height_x','school_x','pos_rk_x','ovr_rk_x','grade_x','text1','text2','text3','text4','draft_year','overall','round','team_abbr']]

In [117]:
scout

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr
0,Bubba Smith,DE,0.0,0.0,Michigan State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,1.0,IND
1,Clinton Jones,RB,0.0,0.0,Michigan State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,2.0,1.0,MIN
2,Steve Spurrier,QB,0.0,0.0,Florida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,3.0,1.0,SF
3,Bob Griese,QB,0.0,0.0,Purdue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,4.0,1.0,MIA
4,George Webster,LB,0.0,0.0,Michigan State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,5.0,1.0,TEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5858,Jeff Gunter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gunter is a versatile edge defender who lines ...,NaN,NaN,NaN,2022.0,NaN,NaN,NaN
5849,Trenton Gill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gill has a strong leg. He led the ACC in punt ...,NaN,NaN,NaN,2022.0,NaN,NaN,NaN
5918,Jesse Luketa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Luketa is a relentless run-defender who has go...,NaN,NaN,NaN,2022.0,NaN,NaN,NaN
5870,Marquis Hayes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hayes is a mauler who plays with good leverage...,NaN,NaN,NaN,2022.0,NaN,NaN,NaN


In [118]:
texts = scout[(scout.text1.notna())|(scout.text2.notna())|(scout.text3.notna())|(scout.text4.notna())]

In [119]:
texts['report'] = None

C:\Users\capta\AppData\Local\Temp\ipykernel_16320\982957188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts['report'] = None


In [127]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5047 entries, 7375 to 5967
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  5047 non-null   object 
 1   pos_abbr_x     4824 non-null   object 
 2   weight_x       4824 non-null   float64
 3   height_x       4824 non-null   float64
 4   school_x       4824 non-null   object 
 5   pos_rk_x       4708 non-null   float64
 6   ovr_rk_x       4129 non-null   float64
 7   grade_x        4714 non-null   float64
 8   text1          3215 non-null   object 
 9   text2          3789 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     5046 non-null   float64
 13  overall        3981 non-null   float64
 14  round          3981 non-null   float64
 15  team_abbr      3981 non-null   object 
 16  report         3216 non-null   object 
dtypes: float64(8), object(9)
memory usage: 838.8+ KB


### Remove Post-Draft Analysis text from scouting report

In [121]:
texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("How he fits")[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_16320\533545784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("How he fits")[0])
C:\Users\capta\AppData\Local\Temp\ipykernel_16320\533545784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].ap

### Remove "What he brings" header from scouting report

In [122]:
texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("What he brings")[1])

C:\Users\capta\AppData\Local\Temp\ipykernel_16320\3585158994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("What he brings")[1])
C:\Users\capta\AppData\Local\Temp\ipykernel_16320\3585158994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.

In [123]:
texts[(texts.text2.str.contains("<a h",na=False) & texts.text1.isna()) & (texts.draft_year > 2008)]

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr,report
8561,Matthew Stafford,QB,225.0,74.250,Georgia,1.0,7.0,96.0,NaN,Stafford has very good upside. He has a very s...,NaN,NaN,2009.0,1.0,1.0,DET,None
8565,Mark Sanchez,QB,227.0,74.125,USC,2.0,8.0,95.0,NaN,Sanchez doesn't have as much arm strength or t...,NaN,NaN,2009.0,5.0,1.0,NYJ,None
8570,Michael Crabtree,WR,215.0,73.375,Texas Tech,1.0,3.0,96.0,NaN,Despite leaving school after his redshirt soph...,NaN,NaN,2009.0,10.0,1.0,SF,None
8571,Aaron Maybin,DE,249.0,75.750,Penn State,1.0,6.0,96.0,NaN,Maybin has the quick first step and the athlet...,NaN,NaN,2009.0,11.0,1.0,BUF,None
8572,Knowshon Moreno,RB,217.0,70.625,Georgia,1.0,12.0,94.0,NaN,We place a high value on competitiveness when ...,NaN,NaN,2009.0,12.0,1.0,DEN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9741,Alec Ogletree,ILB,242.0,74.500,Georgia,1.0,19.0,92.0,NaN,:</b> Ogletree is a freakish athlete who on ta...,NaN,NaN,2013.0,30.0,1.0,STL,None
9742,Travis Frederick,C,312.0,75.625,Wisconsin,1.0,71.0,78.0,NaN,":</b> Frederick is a nasty, versatile interior...",NaN,NaN,2013.0,31.0,1.0,DAL,None
9743,Matt Elam,S,208.0,69.875,Florida,2.0,27.0,89.0,NaN,:</b> The biggest knock on Elam is his lack of...,NaN,NaN,2013.0,32.0,1.0,BAL,None
10338,Dante Fowler Jr.,DE,261.0,74.625,Florida,2.0,4.0,93.0,NaN,:</strong> Fowler is the best all-around edge ...,NaN,NaN,2015.0,3.0,1.0,JAX,None


In [124]:
texts.loc[(texts.text1.notna()),'report'] = texts.loc[(texts.text1.notna())].text1 

In [125]:
texts.loc[(texts.text4.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text4.notna())&(texts.text1.isna())].text4

In [126]:
texts.loc[(texts.report.isna())&(texts.text1.isna())] #= texts.loc[(texts.text4.notna())&(texts.text1.isna())].text2

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr,report
7406,Logan Mankins,OG,307.0,0.000,Fresno State,NaN,NaN,NaN,NaN,The Patriots lost OG Joe Andruzzi to free agen...,NaN,NaN,2005.0,32.0,1.0,NE,None
8429,Reggie Corner,CB,181.0,69.000,Akron,24.0,NaN,40.0,NaN,Corner has good athletic ability and top-end s...,NaN,NaN,2008.0,114.0,4.0,BUF,None
8450,Josh Sitton,OT,320.0,75.625,UCF,45.0,NaN,20.0,NaN,Sitton has excellent size but lacks ideal expl...,NaN,NaN,2008.0,135.0,4.0,GB,None
8455,Brandon Carr,CB,204.0,72.000,Grand Valley State,54.0,NaN,20.0,NaN,Carr has good size for a corner. He has good a...,NaN,NaN,2008.0,140.0,5.0,KC,None
8469,Kroy Biermann,OLB,246.0,74.875,Montana,32.0,NaN,30.0,NaN,"Biermann played defensive end in college, but ...",NaN,NaN,2008.0,154.0,5.0,ATL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12468,Joseph Charlton,P,195.0,76.500,South Carolina,2.0,202.0,48.0,NaN,Charlton is a right-footed punter who owns the...,NaN,NaN,2020.0,NaN,NaN,NaN,None
12475,Benito Jones,DT,316.0,73.000,Ole Miss,18.0,219.0,44.0,NaN,"Jones is a three-year starter with good size, ...",NaN,NaN,2020.0,NaN,NaN,NaN,None
12480,Josiah Coatney,DT,308.0,75.375,Ole Miss,19.0,225.0,42.0,NaN,"Coatney has average size, short arms and below...",NaN,NaN,2020.0,NaN,NaN,NaN,None
12520,Rico Dowdle,RB,213.0,71.250,South Carolina,29.0,289.0,32.0,NaN,Dowdle is a well-built back with average speed...,NaN,NaN,2020.0,NaN,NaN,NaN,None


In [19]:
missing = texts[texts.report.isna()]
missing.draft_year.value_counts()

2015.0    247
2014.0    243
2009.0    239
2010.0    236
2013.0    236
2012.0    233
2011.0    232
2019.0    132
2020.0     31
2008.0     30
2016.0     26
2005.0      4
2018.0      1
Name: draft_year, dtype: int64

In [30]:
missing[missing.draft_year == 2022]

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr,report


In [11]:
texts.loc[(texts.text2.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text2.notna())&(texts.text1.isna())].text2 

In [12]:
texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna()),'report'] = texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna())].text3 

In [13]:
texts.loc[(texts.report.isna()),'report'] = texts.loc[(texts.report.isna())].text1

In [14]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4883 entries, 7375 to 12904
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  4883 non-null   object 
 1   pos_abbr_x     4883 non-null   object 
 2   weight_x       4883 non-null   int64  
 3   height_x       4883 non-null   float64
 4   school_x       4883 non-null   object 
 5   pos_rk_x       4763 non-null   float64
 6   ovr_rk_x       4165 non-null   float64
 7   grade_x        4769 non-null   float64
 8   text1          2992 non-null   object 
 9   text2          3848 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     4882 non-null   float64
 13  overall        4040 non-null   float64
 14  round          4040 non-null   float64
 15  team_abbr      4040 non-null   object 
 16  report         4883 non-null   object 
dtypes: float64(7), int64(1), object(9)
memory usage:

In [28]:
texts.reset_index(inplace=True)

In [30]:
texts.drop(['index'],axis=1,inplace=True)

C:\Users\capta\AppData\Local\Temp\ipykernel_16664\3926660189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.drop(['index'],axis=1,inplace=True)


In [42]:
texts.report[0]

"Nephew of Michigan State head coach John L. Smith.  Alex attempted only five passes as a freshman reserve QB in 2002.  He took over the starting job three games into his sophomore season vs. Cal in 2003 and wound up completing 65-percent of his passes for 2,247 yards with 15 touchdowns compared to just three interceptions.   He also rushed for 452 yards in 2003.  As a junior in 2004, Smith started all 12 games, went undefeated as a starter and led the Utes to a Fiesta Bowl win over Pittsburgh.  Prior to declaring himself available for the 2005 NFL draft, Smith completed 66.1-percent of his passes for 2,624 yards with 28 TD's and four INT's, while also rushing for another 563 yards and 10 TD's.  Smith could have used another year at the collegiate level in order to bulk up and improve his overall strength, but his decision makes sense following a nearly perfect junior season and with head coach Urban Meyer leaving for Florida.  Smith still needs to get bigger and he lacks ideal arm str

In [43]:
report = texts.report

In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.stem import snowball

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\capta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stop_words_list = stopwords.words('english')
stemmer = snowball.EnglishStemmer()

In [64]:
def clean (text, tokenizer, stop_words_list,stemmer):
    lower_text = text.lower()
    token_text = tokenizer.tokenize(lower_text)
    stop_text = [word for word in token_text if word not in stop_words_list]
    stem_text = [stemmer.stem(word) for word in stop_text]
    return stem_text
    

In [66]:
text_data = report.apply(lambda x: clean(x,tokenizer, stop_words_list,stemmer))

In [67]:
text_data

0       [nephew, michigan, state, head, coach, john, s...
1       [brown, play, second, fiddl, carnel, william, ...
2       [edward, father, stanley, play, michigan, nfl,...
3       [benson, draft, los, angel, dodger, 12th, roun...
4       [william, start, two, game, play, nine, 2001, ...
                              ...                        
4878    [jone, big, strong, enough, hold, ground, poin...
4879    [carter, unders, fullback, use, back, type, ls...
4880    [adam, averag, size, top, end, speed, play, ba...
4881    [stiner, interchang, safeti, averag, size, spe...
4882    [imatorbhebh, big, target, averag, speed, good...
Name: report, Length: 4883, dtype: object

In [69]:
report['processed_report'] = text_data

C:\Users\capta\AppData\Local\Temp\ipykernel_16664\1503559796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['processed_report'] = text_data
C:\Users\capta\AppData\Local\Temp\ipykernel_16664\1503559796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['processed_report'] = text_data


In [70]:
report.processed_report

0       [nephew, michigan, state, head, coach, john, s...
1       [brown, play, second, fiddl, carnel, william, ...
2       [edward, father, stanley, play, michigan, nfl,...
3       [benson, draft, los, angel, dodger, 12th, roun...
4       [william, start, two, game, play, nine, 2001, ...
                              ...                        
4878    [jone, big, strong, enough, hold, ground, poin...
4879    [carter, unders, fullback, use, back, type, ls...
4880    [adam, averag, size, top, end, speed, play, ba...
4881    [stiner, interchang, safeti, averag, size, spe...
4882    [imatorbhebh, big, target, averag, speed, good...
Name: report, Length: 4883, dtype: object

In [68]:
cv = CountVectorizer(stop_words='english',min_df=.05,max_df=.9,)
cv.fit_transform(text_data)

AttributeError: 'list' object has no attribute 'lower'

In [61]:
cv.get_feature_names_out()

array(['10', '11', '12', '13', '14', '15', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2015', '2016', '2017', '2019', '2020',
       '40', 'ability', 'adequate', 'alt', 'analysis', 'appeared',
       'april', 'arm', 'arms', 'assets', 'athlete', 'athletic',
       'athleticism', 'average', 'backup', 'balance', 'ball', 'best',
       'better', 'big', 'bit', 'blocker', 'blocks', 'body', 'border',
       'bowl', 'br', 'brings', 'bulk', 'burst', 'career', 'catch',
       'catches', 'caught', 'chance', 'class', 'college', 'com',
       'combination', 'combine', 'comes', 'concerns', 'contribute',
       'corner', 'cover', 'coverage', 'day', 'defender', 'defenders',
       'defense', 'defensive', 'depth', 'develop', 'did', 'does', 'doesn',
       'draft', 'durability', 'early', 'edge', 'effective', 'effort',
       'elite', 'em', 'end', 'espn', 'evaluation', 'excellent',
       'experience', 'explosive', 'field', 'final', 'finished',
       'finishing', 'fit', 'fits', 'flas

In [18]:
#texts.to_parquet('./Data/draft_reports.parquet')